# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
import joblib
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.classify.scikitlearn import SklearnClassifier
from textblob import TextBlob
from bs4 import BeautifulSoup

import pickle
import re, string
from collections import Counter
from string import punctuation
import tweepy
from tweepy import OAuthHandler
import json
from wordcloud import WordCloud

import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()

from IPython.display import IFrame
import folium
from folium import plugins
from folium.plugins import MarkerCluster, FastMarkerCluster, HeatMapWithTime


# Process the train data

In [5]:
#label 0-negative, 2-means positive, 4-neutral
data = pd.read_csv("trainingdata.csv",skip_blank_lines=True,encoding = "latin") 
data

,label,tweet
0,0,kenapa
1,0,solange knowles marah kepada saya
2,0,berbunyi pecah membuat saya menangis
3,0,adalah kejutan Kate tidak memenangi pelatih
4,0,ketawa kuat. . . saya tidak percaya saya terle...
...,...,...
1554058,2,@spielpIatz hy sayang kenalan yu
1554059,2,/wal aku cantik lah kelonya ku cantik?
1554060,2,Wtb slogan! Stray kids hyunjin atau IN drop do...
1554061,2,1. 2017 - Anem\r\n2. 2017 - Keyla\r\n3. 2018 -...


In [6]:
data2 = pd.DataFrame(df)
data2.drop(data2.index[0:160000],0, inplace=True)

In [7]:
# check the number of positive vs. negative tagged sentences
negatives = data2['label'][data2.label == 0]
positives = data2['label'][data2.label == 2]
neutral = data2['label'][data2.label == 4]

print('number of negative tagged sentences is: {}'.format(len(negatives)))
print('number of positive tagged sentences is: {}'.format(len(positives)))
print('number of neutral tagged sentences  is: {}'.format(len(neutral)))
print('total length of the data is:            {}'.format(data2.shape[0]))

number of negative tagged sentences is: 637760
number of positive tagged sentences is: 743846
number of neutral tagged sentences  is: 12457
total length of the data is:            1394063


In [8]:
data = pd.DataFrame(data2)
data.drop(data.index[1000000:1106086],0, inplace=True)

In [15]:
# check the number of positive vs. negative tagged sentences
negatives = data['label'][data.label == 0]
positives = data['label'][data.label == 2]
neutral = data['label'][data.label == 4]

print('number of negative tagged sentences is: {}'.format(len(negatives)))
print('number of positive tagged sentences is: {}'.format(len(positives)))
print('number of neutral tagged sentences  is: {}'.format(len(neutral)))
print('total length of the data is:            {}'.format(data.shape[0]))

number of negative tagged sentences is: 637759
number of positive tagged sentences is: 637760
number of neutral tagged sentences  is: 12457
total length of the data is:            1287976


In [16]:
data = data.dropna()

In [17]:
data.isnull().values.any()

False

# Natural Language Processing (NLP)

###### Preprocessing 1:  Clean tweet text by removing links, special characters

In [18]:
def text_cleaning(tweet):
    tweet = tweet.lower() #convert text to lower-case
    tweet = re.sub(r'\&\w*;', '',tweet) # remove HTML special entities (e.g. &amp;)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'\$\w*', '', tweet) # remove tickers
    tweet = re.sub(r'#\w*', '', tweet) # remove hashtags
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet) # remove hyperlinks
    tweet = tweet.lstrip(' ') # remove single space remaining at the front of the tweet.
    tweet = re.sub(r'\s\s+', ' ', tweet) # remove whitespace (including new line characters) 
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet) # remove words with 2 or fewer letters 
    tweet = tweet.lstrip(' ') # remove single space remaining at the front of the tweet.
  
    # remove characters beyond Basic Multilingual Plane (BMP) of unicode (contains characters for almost all modern languages, and a large number of symbols):
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

    # clean dataframe's text column
    data['tweet'] = data['tweet'].apply(text_cleaning)
    # preview some cleaned tweets
    data['tweet'].head()

In [19]:
print(data.dtypes)

label    int64 
tweet    object
dtype: object


In [20]:
# drop duplicates
data = data.drop_duplicates('tweet')
data.shape

(1185758, 2)

###### Pre-processing 2: Tokenize without out Stop-Words

In [21]:
with open('stopwords-ms.txt','r') as f2:
   b=f2.read().split()
   print(' '.join(x.lower() for x in b))

abdul abdullah acara ada adalah ahmad air akan akhbar akhir aktiviti alam amat amerika anak anggota antara antarabangsa apa apabila april as asas asean asia asing atas atau australia awal awam bagaimanapun bagi bahagian bahan baharu bahawa baik bandar bank banyak barangan baru baru-baru bawah beberapa bekas beliau belum berada berakhir berbanding berdasarkan berharap berikutan berjaya berjumlah berkaitan berkata berkenaan berlaku bermula bernama bernilai bersama berubah besar bhd bidang bilion bn boleh bukan bulan bursa cadangan china dagangan dalam dan dana dapat dari daripada dasar datang datuk demikian dengan depan derivatives dewan di diadakan dibuka dicatatkan dijangka diniagakan dis disember ditutup dolar dr dua dunia ekonomi eksekutif eksport empat enam faedah feb global hadapan hanya harga hari hasil hingga hubungan ia iaitu ialah indeks india indonesia industri ini islam isnin isu itu jabatan jalan jan jawatan jawatankuasa jepun jika jualan juga julai jumaat jumlah jun juta ka

In [22]:
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nanan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
# tokenize helper function
def process_text(text):
    nopunc = [char for char in list(text) if char not in string.punctuation] # check characters to see if they are in punctuation
    nopunc = ''.join(nopunc) # join the characters again to form the string
    return [word for word in nopunc.lower().split() if word.lower() not in b] # remove any stopwords

def remove_words(word_list):
    remove = ['com','pic','twitter','...','“','”','’','…']
    return [w for w in word_list if w not in remove]

# tokenize message column and create a column for tokens
data = data.copy()
data['tokens'] = data['tweet'].apply(process_text)
data['words'] = data['tokens'].apply(remove_words)
data.head()

,label,tweet,tokens,words
160000,0,hari ini. entah bagaimana ia tidak kelihatan sangat penting. hidup agak sucks sekarang. saya perlukan beberapa rancangan.,"[entah, bagaimana, kelihatan, sangat, hidup, agak, sucks, perlukan, rancangan]","[entah, bagaimana, kelihatan, sangat, hidup, agak, sucks, perlukan, rancangan]"
160001,0,"marah. saya sangat tidak suka ineptitude dan cenderung berpandangan yang keras kepala. tuhan yang sayang, tolong saya daripada membunuh sesiapa sahaja di tempat kerja","[marah, sangat, suka, ineptitude, cenderung, berpandangan, keras, kepala, tuhan, sayang, tolong, membunuh, sesiapa]","[marah, sangat, suka, ineptitude, cenderung, berpandangan, keras, kepala, tuhan, sayang, tolong, membunuh, sesiapa]"
160002,0,"ia figgers. satu hari minggu ini saya boleh pulang pada pukul 6 petang, dan lalu lintas menghisap sekarang terperangkap di pejabat yang merasa bersalah kerana tidak bekerja. lt","[figgers, pulang, 6, lalu, lintas, menghisap, terperangkap, merasa, bersalah, bekerja, lt]","[figgers, pulang, 6, lalu, lintas, menghisap, terperangkap, merasa, bersalah, bekerja, lt]"
160003,0,"pagi esok. . . matahari, tidak hujan setakat ini. . . sakit kepala alergi lebih baik, tetapi tidak benar-benar hilang. . . akan berehat hari ini. . . ingin saya boleh duduk di luar","[esok, matahari, hujan, setakat, sakit, kepala, alergi, benarbenar, hilang, berehat, ingin, duduk]","[esok, matahari, hujan, setakat, sakit, kepala, alergi, benarbenar, hilang, berehat, ingin, duduk]"
160004,0,8. 5 jam lagi. . . .,"[8, 5, jam]","[8, 5, jam]"


In [24]:
# style 1 tokens
data['tokens'].head(10)

160000    [entah, bagaimana, kelihatan, sangat, hidup, agak, sucks, perlukan, rancangan]                                     
160001    [marah, sangat, suka, ineptitude, cenderung, berpandangan, keras, kepala, tuhan, sayang, tolong, membunuh, sesiapa]
160002    [figgers, pulang, 6, lalu, lintas, menghisap, terperangkap, merasa, bersalah, bekerja, lt]                         
160003    [esok, matahari, hujan, setakat, sakit, kepala, alergi, benarbenar, hilang, berehat, ingin, duduk]                 
160004    [8, 5, jam]                                                                                                        
160005    [menjual, festival, pekan, twitter]                                                                                
160006    [ohh, sedih, suka]                                                                                                 
160007    [sooo, letih, ingin, bercakap, seseorang, terlepas, begitu]                                                 

###### Pre-processing 3: Feature Extraction

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
# vectorize (convert each message which is represented by a list of tokens into a vector that a machine learning model can understand)
# converts a collection of text documents to a matrix of token counts
bow_transformer = CountVectorizer(analyzer=process_text).fit(data['tweet']) 
# print total number of vocab words
print(len(bow_transformer.vocabulary_))

887336


In [26]:
bow_transformer.vocabulary_

{'entah': 163202,
 'bagaimana': 66657,
 'kelihatan': 584569,
 'sangat': 769584,
 'hidup': 213136,
 'agak': 27115,
 'sucks': 809281,
 'perlukan': 721472,
 'rancangan': 747536,
 'marah': 631592,
 'suka': 810154,
 'ineptitude': 548091,
 'cenderung': 108669,
 'berpandangan': 82507,
 'keras': 587761,
 'kepala': 586912,
 'tuhan': 845602,
 'sayang': 772524,
 'tolong': 840396,
 'membunuh': 641973,
 'sesiapa': 784566,
 'figgers': 176382,
 'pulang': 737267,
 '6': 15045,
 'lalu': 608905,
 'lintas': 617257,
 'menghisap': 646784,
 'terperangkap': 831181,
 'merasa': 650534,
 'bersalah': 82992,
 'bekerja': 76057,
 'lt': 621291,
 'esok': 165143,
 'matahari': 634706,
 'hujan': 538116,
 'setakat': 784805,
 'sakit': 767124,
 'alergi': 36454,
 'benarbenar': 77225,
 'hilang': 213742,
 'berehat': 79639,
 'ingin': 548720,
 'duduk': 153597,
 '8': 16947,
 '5': 13854,
 'jam': 559373,
 'menjual': 648319,
 'festival': 175574,
 'pekan': 713229,
 'twitter': 848505,
 'ohh': 698797,
 'sedih': 776179,
 'sooo': 801990,

In [27]:
# example of vectorized text
sample_tweet = data['tweet']
print(sample_tweet)
print('\n')
# vector representation
bow_sample = bow_transformer.transform([sample_tweet])
print(bow_sample)
print('\n')

160000     hari ini. entah bagaimana ia tidak kelihatan sangat penting. hidup agak sucks sekarang. saya perlukan beberapa rancangan.                                                           
160001     marah. saya sangat tidak suka ineptitude dan cenderung berpandangan yang keras kepala. tuhan yang sayang, tolong saya daripada membunuh sesiapa sahaja di tempat kerja              
160002     ia figgers. satu hari minggu ini saya boleh pulang pada pukul 6 petang, dan lalu lintas menghisap sekarang terperangkap di pejabat yang merasa bersalah kerana tidak bekerja. lt    
160003     pagi esok. . . matahari, tidak hujan setakat ini. . . sakit kepala alergi lebih baik, tetapi tidak benar-benar hilang. . . akan berehat hari ini. . . ingin saya boleh duduk di luar
160004     8. 5 jam lagi. . . .                                                                                                                                                                
                   ...                  

In [28]:
# transform the entire DataFrame of messages
messages_bow = bow_transformer.transform(data['tweet'])

# check out the bag-of-words counts for the entire corpus as a large sparse matrix
print('Shape of Sparse Matrix: ', messages_bow.shape)
print('Amount of Non-Zero occurences: ', messages_bow.nnz)

Shape of Sparse Matrix:  (1185758, 887336)
Amount of Non-Zero occurences:  10482994


###### Term Frequency, Inverse Document Frequency

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf_sample = tfidf_transformer.transform(bow_sample)
print(tfidf_sample)

  (0, 887335)	5.449045700694676e-05
  (0, 887334)	1.0898091401389353e-05
  (0, 887332)	1.0898091401389353e-05
  (0, 887331)	1.0898091401389353e-05
  (0, 887330)	1.0898091401389353e-05
  (0, 887328)	1.0898091401389353e-05
  (0, 887327)	1.0898091401389353e-05
  (0, 887326)	1.0898091401389353e-05
  (0, 887325)	1.0898091401389353e-05
  (0, 887324)	1.0898091401389353e-05
  (0, 887323)	1.0898091401389353e-05
  (0, 887322)	1.0898091401389353e-05
  (0, 887321)	1.0898091401389353e-05
  (0, 887320)	4.971435418459469e-05
  (0, 887319)	2.1177855762180886e-05
  (0, 887318)	1.0898091401389353e-05
  (0, 887317)	1.0898091401389353e-05
  (0, 887316)	2.1177855762180886e-05
  (0, 887314)	3.110871861769321e-05
  (0, 887313)	1.0898091401389353e-05
  (0, 887312)	1.0898091401389353e-05
  (0, 887311)	1.0898091401389353e-05
  (0, 887310)	1.0898091401389353e-05
  (0, 887309)	1.0898091401389353e-05
  (0, 887308)	1.0898091401389353e-05
  :	:
  (0, 99)	1.0898091401389353e-05
  (0, 95)	2.925373749589976e-05
  (0, 9

In [30]:
# to transform the entire bag-of-words corpus
tweet_tfidf = tfidf_transformer.transform(messages_bow)
print(tweet_tfidf.shape)

(1185758, 887336)


## Model Training and Evaluation

In [31]:
#import the necessary libraries for dataset preparation, feature engineering, model training
from sklearn import model_selection, preprocessing, metrics, linear_model, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline

In [32]:
#split the dataset into training and test datasets 
X_train, X_test, y_train, y_test = model_selection.train_test_split(data['tweet'],data['label'], test_size=0.2)
#label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [33]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(data['tweet'])
xtrain_tfidf =  tfidf_vect.transform(X_train)
xtest_tfidf =  tfidf_vect.transform(X_test)

In [34]:
#Return the f1 Score
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    return metrics.f1_score(y_test,predictions,average = 'weighted')

In [35]:
from sklearn.svm import LinearSVC
from imblearn.pipeline import Pipeline

In [36]:
from sklearn import ensemble

In [39]:
#Random Over Sampling
ros = RandomOverSampler(random_state=777)
ros_xtrain_tfidf, ros_ytrain = ros.fit_sample(xtrain_tfidf, y_train)
accuracyROS = train_model(LinearSVC(class_weight='balanced', penalty='l2', loss='hinge', max_iter=1000000),ros_xtrain_tfidf, ros_ytrain, xtest_tfidf)


In [56]:
print ("SVM ROS: ", accuracyROS)

SVM ROS:  0.8532040628735464


In [47]:
# save model to current working directory
joblib.dump(accuracyROS, "twitter_sentiment_ROS.pkl")

['twitter_sentiment_ROS.pkl']

In [ ]:
sm = SMOTE()
model = Pipeline([
    ('sampling', SMOTE()),
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', LinearSVC(class_weight='balanced', penalty='l2', loss='hinge', max_iter=100000))
])

sm_xtrain_tfidf, sm_ytrain = sm.fit_sample(xtrain_tfidf, y_train)
accuracySMOTE = train_model(model, sm_xtrain_tfidf, sm_ytrain, xtest_tfidf)
print ("SVC SMOTE, WordLevel TFIDF: ", accuracySMOTE)

In [71]:
from imblearn.pipeline import Pipeline

In [ ]:
from sklearn.svm import LinearSVC
from imblearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)

model = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', LinearSVC(class_weight='balanced', penalty='l2', loss='hinge', random_state=777, max_iter=100000))
])

model.fit(X_train,y_train)

In [ ]:
y_preds = model.predict(X_test)

print('accuracy score: ',accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n',confusion_matrix(y_test,y_preds))
print('\n')
print(classification_report(y_test, y_preds))

#### Random Forest Classifier

In [16]:
# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)


# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', RandomForestClassifier()),  
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__n_estimators': [20, 50],
                'clf__max_features': ['auto'],
                'clf__max_depth' : [4,5,6],
                'clf__criterion' :['gini', 'entropy'],
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 125.2min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 138.7min finished



Best Model: 0.533935 using {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 6, 'clf__max_features': 'auto', 'clf__n_estimators': 20, 'tfidf__use_idf': False}


Mean: 0.524345 Stdev:(0.005149) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 20, 'tfidf__use_idf': True}
Mean: 0.526015 Stdev:(0.008175) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 20, 'tfidf__use_idf': False}
Mean: 0.519057 Stdev:(0.000766) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 50, 'tfidf__use_idf': True}
Mean: 0.518440 Stdev:(0.000366) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 50, 'tfidf__use_idf': False}
Mean: 0.528067 Stdev:(0.008229) with: {'bow__

#### Naive Bayes

In [19]:
# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)


# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', MultinomialNB()),  
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__alpha': (1e-2, 1e-3),
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 65.7min finished



Best Model: 0.836509 using {'bow__ngram_range': (1, 2), 'clf__alpha': 0.01, 'tfidf__use_idf': False}


Mean: 0.803009 Stdev:(0.001064) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.818192 Stdev:(0.001022) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.799113 Stdev:(0.000957) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.812924 Stdev:(0.001147) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.001, 'tfidf__use_idf': False}
Mean: 0.823180 Stdev:(0.001459) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.836509 Stdev:(0.001448) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.810987 Stdev:(0.001503) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.825470 Stdev:(0.001383) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.001, 'tfidf__use_idf': False}


In [21]:
# save best model to current working directory
joblib.dump(grid, "twitter_sentiment_NB.pkl")

['twitter_sentiment_NB.pkl']

#### SVM Classifier

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier 

# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)

# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', SGDClassifier(class_weight='balanced')),  
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__loss' : ["log", "squared_hinge", "modified_huber"],
                'clf__alpha': (1e-3, 1e-4),
                'clf__penalty' : ["l2", "none"],
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.6min


In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)

# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced', max_iter=100000))) 
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1, 1), (1, 2), (1,3)],
                'tfidf__use_idf': (True, False),
                'clf__estimator__loss' : ["hinge", "squared_hinge"],
                'clf__estimator__penalty' : ["l2", "none"], 
                'clf__estimator__tol' : [1e-4, 1e-5]
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, verbose=1, n_jobs= 8)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 130.0min


In [ ]:
# save best model to current working directory
joblib.dump(grid, "twitter_sentiment_LSVM_balanced_bm.pkl")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import auc,roc_curve

In [ ]:
model_LSVC = joblib.load("twitter_sentiment_LSVM_balanced_bm.pkl" )

# get predictions from best model above
y_preds = model_LSVC.predict(X_test)

print('accuracy score: ',accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n',confusion_matrix(y_test,y_preds))
print('\n')
print(metrics.classification_report(y_test, y_preds))

In [ ]:
! pip install -q scikit-plot

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(
    y_test, 
    y_preds,
    figsize=(10,10))

#### Stacking Classifier

In [19]:
from sklearn.base import BaseEstimator
class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = SGDClassifier(),
    ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [ ]:
# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)


# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', ClfSwitcher()),  
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'clf__estimator': [SGDClassifier()],
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__estimator__loss' : ["squared_hinge", "modified_huber"],
                'clf__estimator__alpha' : [0.001, 0.01, 0.1],
                'clf__estimator__penalty' : ["l2"],
            },
            {
                'clf__estimator': [MultinomialNB()],
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__estimator__alpha': (1e-2, 1e-3),
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=5, param_grid=parameters, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 32 candidates, totalling 160 fits


In [ ]:
# save best model to current working directory
joblib.dump(grid, "twitter_sentiment_ensemble_NB_SGD.pkl")

#### Model Evaluation

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import auc,roc_curve

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)

In [16]:
model_LSVC = joblib.load("twitter_sentiment_LSVM_balanced_final.pkl" )

# get predictions from best model above
y_preds = model_LSVC.predict(X_test)

print('accuracy score: ',accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n',confusion_matrix(y_test,y_preds))
print('\n')
print(metrics.classification_report(y_test, y_preds))

C:\Users\nanan\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator CountVectorizer from version 0.22.2.post1 when using version 0.23.0. This might lead to breaking code or invalid results. Use at your own risk.

C:\Users\nanan\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 0.23.0. This might lead to breaking code or invalid results. Use at your own risk.

C:\Users\nanan\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator LinearSVC from version 0.22.2.post1 when using version 0.23.0. This might lead to breaking code or invalid results. Use at your own risk.

C:\Users\nanan\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator OneVsRestClassifier from version 0.22.2.post1 when using version 0.23.0. This might lead to breaking code or invalid results. Use at your own 

accuracy score:  0.9481164303793537


confusion matrix: 
 [[119360   7508    205]
 [  4694 122914    350]
 [   214    394   1957]]


              precision    recall  f1-score   support

           0       0.96      0.94      0.95    127073
           2       0.94      0.96      0.95    127958
           4       0.78      0.76      0.77      2565

    accuracy                           0.95    257596
   macro avg       0.89      0.89      0.89    257596
weighted avg       0.95      0.95      0.95    257596

